In [2]:
'''

function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);


function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

'\n\nfunction ConnectButton(){\n    console.log("Connect pushed"); \n    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() \n}\nsetInterval(ConnectButton,60000);\n\n\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}\nsetInterval(ClickConnect,60000)\n'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!cat /proc/meminfo

MemTotal:       26751700 kB
MemFree:        25502976 kB
MemAvailable:   25716984 kB
Buffers:           20560 kB
Cached:           464808 kB
SwapCached:            0 kB
Active:           657548 kB
Inactive:         318536 kB
Active(anon):     471712 kB
Inactive(anon):      376 kB
Active(file):     185836 kB
Inactive(file):   318160 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               644 kB
Writeback:             0 kB
AnonPages:        490940 kB
Mapped:           116148 kB
Shmem:               928 kB
Slab:             123520 kB
SReclaimable:      66848 kB
SUnreclaim:        56672 kB
KernelStack:        4720 kB
PageTables:         7164 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375848 kB
Committed_AS:    3254808 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1888 kB
AnonHugePages:   

In [5]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [6]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("GSAR")

R[write to console]: Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

R[write to console]: Installing package(s) 'GSAR'

R[write to console]: trying URL 'https://bioconductor.org/packages/3.10/bioc/src/contrib/GSAR_1.20.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2185168 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

In [7]:
%%R
library(tidyverse)
library(MASS)
library(GSAR)
library(parallel)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: 
Attaching package: ‘MASS’


R[write to console]: The following object is masked from ‘package:dplyr’:

    select


R[write to console]: Loading required package: igraph

R[write to console]: 
Attaching package: ‘igraph’


R[write to console]: The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


R[write to console]: The following objects are masked from ‘package:purrr’:

    compose, simplify


R[write to console]: The following object is masked from ‘package:tidyr’:

    crossing


R[w

In [8]:
%%R

# Simplex sample generator
simplex_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n-1,0,1)  %>%  
                    sort(decreasing=FALSE)
        sample  <- append(0.0, sample)  %>% 
                    append(1.0)
        sample  <- sample[-1] - sample
        sample  <- sample[1:n]

        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# Genberate hypercube
hyper_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n,-1,1) 
        sample  <- sample[1:n]
        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# MST for simplex
mst <- function(file_, nsamp=10000, perm=50){
    # Read files with samples
    df_  <- read_csv(file_)  %>%  
    dplyr::select(starts_with('x_'))  %>% 
    sample_n(size=nsamp, replace=FALSE)
    # Get dim of the read file
    n_rows  <- dim(df_)[1]
    n_cols  <- dim(df_)[2]
        if (n_rows > (nsamp-1)){
        simplex_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
        
        df_  <- df_  %>% 
                data.matrix()   %>% 
                unname()
        
        df_  <- rbind(df_, simplex_sample)  
        df_  <- df_ %>% 
                aperm(c(2,1)) 
        result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                         nperm=perm, pvalue.only=FALSE)
        z_df  <- tibble(dim=n_cols, z=result$statistic, 
                        p_value=result$p.value,
                             nsamp=nsamp)
        gc()
        return(z_df)
        
    }
    
}


In [9]:
%%R
path  <- '/content/drive/My Drive/mhar_paper_samples_2/'
set.seed(5)
files <- list.files(path)
files  <- paste(path, files, sep='')

# Extract dimenions
m <- regexpr("%[0-9]+.csv", files, perl=TRUE)
dimension <- regmatches(files, m)
m <- regexpr("[0-9]+", dimension, perl=TRUE)
dimension <- regmatches(dimension, m)
dimension

#Extract figure
m <- regexpr("[0-9]+_[a-z]+%", files, perl=TRUE)
figure <- regmatches(files, m)
m <- regexpr("[a-z]+", figure, perl=TRUE)
figure <- regmatches(figure, m)
figure

# Extract seed
m <- regexpr("/[0-9]+_", files, perl=TRUE)
sed <- regmatches(files, m)
m <- regexpr("[0-9]+", sed, perl=TRUE)
sed <- regmatches(sed, m)

df_simulations <- tibble(file=files, n=dimension, figure=figure, seed=sed)

In [10]:
%%R
df_simulations

# A tibble: 81 x 4
   file                                                     n     figure   seed 
   <chr>                                                    <chr> <chr>    <chr>
 1 /content/drive/My Drive/mhar_paper_samples_2/100_hyperc… 3     hypercu… 100  
 2 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 15    hypercu… 1064 
 3 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 25    hypercu… 1064 
 4 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 3     hypercu… 1064 
 5 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 50    hypercu… 1064 
 6 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 15    simplex  1064 
 7 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 25    simplex  1064 
 8 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 3     simplex  1064 
 9 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 50    simplex  1064 
10 /content/drive/My Drive/mhar_paper_samples_2/1197_hyper… 15    hypercu… 1197 
# … with 

In [11]:
%%R

save_path <- '/content/drive/My Drive/mst_5000/mst_5000_heated_.csv'

# Create random seeds
seeds_ <- sample(1:10000000, 1000, replace=T)
s<- 1


## PArse existing experiments
if(!file.exists(save_path)){
    existance<- tibble(figure=character(),dim=numeric(), z=numeric(), 
                          p_value=numeric(),
                              nsamp=numeric(), file=character(), perm=numeric(),
               seed=numeric()) 
    print('Experiments file does not exists')
    
}else{
    existance <- read.csv(save_path) %>% dplyr::select( -contains('X'))
    print('Loading experiment csv')
}

z_df <- existance

for(f in files){
  # Set hyperparameters
  set.seed(seeds_[s])
  file_name <- f
  perm <- 2
  n_sample <- 5
  print(file_name)
  df_  <- read_csv(file_name)
  df_ <- df_[c(1:n_sample),]
  # Get dim of the read file
  n_rows  <- dim(df_)[1]
  n_cols  <- dim(df_)[2]
  print(n_rows)
  print(n_cols)

  # Check if the experiment already runned
  check_df <- FALSE
  if(f %in% existance$file){
    check_df <- TRUE
  }

  check_df <- !check_df

  if(check_df){
    typ <- 'Error'
      print('Run MST')
    if (grepl('hyper', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
    typ <- 'Hypercube'
  }
  if (grepl('simplex', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
    typ <- 'Simplex'
  }

  df_  <- df_  %>% data.matrix()   %>% unname()
          
  df_  <- rbind(df_, real_sample)  
  df_  <- df_ %>% aperm(c(2,1)) 
  dim(df_)
  result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                          nperm=perm, pvalue.only=FALSE)
  z_df  <- tibble(figure=typ,dim=n_cols, z=result$statistic, 
                          p_value=result$p.value,
                              nsamp=n_sample, file=f, perm=perm, 
                  seed=seeds_[s]) %>% rbind(z_df)
  z_df %>% write.csv(save_path)
  
  }
  #Increment seed index
  s <- s + 1 
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  print('')
  print('')
}




[1] "Experiments file does not exists"
[1] "/content/drive/My Drive/mhar_paper_samples_2/100_hypercube%1000%3.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double()
)



[1] 5
[1] 3
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""
[1] "/content/drive/My Drive/mhar_paper_samples_2/1064_hypercube%1000%15.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double(),
  `10` = col_double(),
  `11` = col_double(),
  `12` = col_double(),
  `13` = col_double(),
  `14` = col_double()
)



[1] 5
[1] 15
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""
[1] "/content/drive/My Drive/mhar_paper_samples_2/1064_hypercube%1000%25.csv"


...

[1] 5
[1] 25
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""
[1] "/content/drive/My Drive/mhar_paper_samples_2/931_simplex%1000%3.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double()
)



[1] 5
[1] 3
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""
[1] "/content/drive/My Drive/mhar_paper_samples_2/931_simplex%1000%50.csv"


R[write to console]: Parsed with column specification:
cols(
  .default = col_double()
)

R[write to console]: See spec(...) for full column specifications.



[1] 5
[1] 50
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""


In [ ]:
%%R

save_path <- '/content/drive/My Drive/mst_5000/mst_5000_initial.csv'

# Create random seeds
seeds_ <- sample(1:10000000, 1000, replace=T)
s<- 1


## PArse existing experiments
if(!file.exists(save_path)){
    existance<- tibble(figure=character(),dim=numeric(), z=numeric(), 
                          p_value=numeric(),
                              nsamp=numeric(), file=character(), perm=numeric(),
               seed=numeric()) 
    print('Experiments file does not exists')
    
}else{
    existance <- read.csv(save_path) %>% dplyr::select( -contains('X'))
    print('Loading experiment csv')
}

z_df <- existance

for(f in files){
  # Set hyperparameters
  set.seed(seeds_[s])
  file_name <- f
  perm <- 50
  n_sample <- 5000
  print(file_name)
  df_  <- read_csv(file_name)
  df_ <- df_[c(1:n_sample),]
  # Get dim of the read file
  n_rows  <- dim(df_)[1]
  n_cols  <- dim(df_)[2]
  print(n_rows)
  print(n_cols)

  # Check if the experiment already runned
  check_df <- FALSE
  if(f %in% existance$file){
    check_df <- TRUE
  }

  check_df <- !check_df

  if(check_df){
    typ <- 'Error'
      print('Run MST')
    if (grepl('hyper', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
    typ <- 'Hypercube'
  }
  if (grepl('simplex', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
    typ <- 'Simplex'
  }

  df_  <- df_  %>% data.matrix()   %>% unname()
          
  df_  <- rbind(df_, real_sample)  
  df_  <- df_ %>% aperm(c(2,1)) 
  dim(df_)
  result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                          nperm=perm, pvalue.only=FALSE)
  z_df  <- tibble(figure=typ,dim=n_cols, z=result$statistic, 
                          p_value=result$p.value,
                              nsamp=n_sample, file=f, perm=perm, 
                  seed=seeds_[s]) %>% rbind(z_df)
  z_df %>% write.csv(save_path)
  
  }
  #Increment seed index
  s <- s + 1 
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  print('')
  print('')
}




[1] "Experiments file does not exists"
[1] "/content/drive/My Drive/mhar_paper_samples_2/100_hypercube%1000%3.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double()
)



[1] 5000
[1] 3
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""
[1] "/content/drive/My Drive/mhar_paper_samples_2/1064_hypercube%1000%15.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double(),
  `10` = col_double(),
  `11` = col_double(),
  `12` = col_double(),
  `13` = col_double(),
  `14` = col_double()
)



[1] 5000
[1] 15
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""
[1] "/content/drive/My Drive/mhar_paper_samples_2/1064_hypercube%1000%25.csv"


R[write to console]: Parsed with column specification:
cols(
  .default = col_double()
)

R[write to console]: See spec(...) for full column specifications.



[1] 5000
[1] 25
[1] "Run MST"
[1] "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
[1] ""
[1] ""
[1] "/content/drive/My Drive/mhar_paper_samples_2/1064_hypercube%1000%3.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double()
)



[1] 5000
[1] 3
[1] "Run MST"
